# A Test for the `InteractiveViewer`

In [ ]:
import json
import os
import random
import time
from pathlib import Path

import importnb
import ipywidgets as W
from rdflib import Graph
from requests_cache import CachedSession

from ipyradiant.query.api import SPARQLQueryFramer

with importnb.Notebook():
    try:
        from .InteractiveViewer import lw, simple_graph, iv
    except ImportError:
        from InteractiveViewer import lw, simple_graph, iv

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automated section below.

In [ ]:
pl = lw.loader

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Tests are implemented as buttons you click.

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
# TODO extend as needed
tests = [
    W.Button(description="simple_graph")
]

In [ ]:
N_CHECKS = 5  # number of times to run the test selection


def _run_test(btn):
    p = btn.description
    
    try:
        timestamp(p, "starting...")
        iv.graph = Graph()
        timestamp(p, "cleaned...")
        assert len(iv.viewer.cytoscape_widget.graph.nodes) == 0
        timestamp(p, f"assigning graph...")
        iv.graph = simple_graph
        assert len(iv.viewer.cytoscape_widget.graph.nodes) > 0
        timestamp(p, f"graph updated...")
        
        timestamp(p, f"verifying node visibility...")
        cyto_graph_nodes = iv.viewer.cytoscape_widget.graph.nodes
        n_nodes = len(cyto_graph_nodes)
        visible_nodes = [node for node in cyto_graph_nodes if 'invisible' not in node.classes]
        n_visible_nodes = len(visible_nodes)
        assert n_nodes == n_visible_nodes
        timestamp(p, f"interactive viewer initialized w/ graph successfully...")
        
        for ii in range(N_CHECKS):
            timestamp(p, f"making selections for iteration #{ii}...")
            # get some number of options from the list (at least one)
            selections = random.sample(
                iv.type_selector.options, 
                k=random.choice(range(1,len(iv.type_selector.options)))
            )
            selection_values = tuple([_[1] for _ in selections])
            # make the selections
            iv.type_selector.value = selection_values

            # get visible nodes
            timestamp(p, f"get visible nodes for iteration #{ii}...")
            visible_nodes = [node for node in cyto_graph_nodes if 'invisible' not in node.classes]
            n_visible_nodes = len(visible_nodes)

            timestamp(p, f"calculating expected number of visible nodes for iteration #{ii}...")
            n_ex_visible = 0
            for node in cyto_graph_nodes:
                if type(node.data['rdf:type']) is tuple:
                    types = set(node.data['rdf:type'])
                else:
                    types = set([node.data['rdf:type']])

                if any([visible_type in types for visible_type in selection_values]):
                    n_ex_visible += 1

            assert n_ex_visible == n_visible_nodes
            timestamp(p, f"expected matches visible for iteration #{ii}...")
        
        timestamp(p, "OK!")
    except Exception as err:
        timestamp(p, "ERROR")
        timestamp(p, err)
        raise Exception(f"{p} failed") from err
        
[d.on_click(_run_test) for d in tests]

## Show the Test Application

In [ ]:
W.VBox([iv, W.HBox([W.Label("Start Test"), *tests])])

# Run Test Automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()